# YOLO vs SAM Karşılaştırmalı Analiz
Bu notebook kısa video ile YOLO ve SAM analizlerini karşılaştırır.

## 1. Sistem Hazırlığı ve Kontrol

In [ ]:
# Sistem kontrolü
import os
import sys
import subprocess
import time
from datetime import datetime

print(f"🖥️  Çalışma dizini: {os.getcwd()}")
print(f"🐍 Python versiyonu: {sys.version}")
print(f"📅 Analiz zamanı: {datetime.now()}")

In [ ]:
# GPU kontrol
!nvidia-smi --query-gpu=name,memory.total,memory.used,utilization.gpu --format=csv

In [ ]:
# PyTorch CUDA kontrolü
import torch
print(f"🔥 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"🔄 CUDA Version: {torch.version.cuda}")

## 2. Video İndirme ve Hazırlık

In [ ]:
# Çalışma dizinini ayarla
os.chdir('/workspace/line48')
print(f"📁 Çalışma dizini: {os.getcwd()}")

# Gerekli dizinleri oluştur
!mkdir -p videos outputs logs
!ls -la

In [ ]:
# Kısa videoyu Google Drive'dan indir
import gdown

video_url = "https://drive.google.com/file/d/1rpgW_pmdMiUp_9_BVkRmXtF7Hon_u1gC/view?usp=share_link"
file_id = "1rpgW_pmdMiUp_9_BVkRmXtF7Hon_u1gC"
output_path = "videos/short_video.mp4"

print(f"📥 Kısa video indiriliyor...")
print(f"🔗 URL: {video_url}")
print(f"📂 Çıktı: {output_path}")

gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

# Dosya kontrolü
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / (1024*1024)  # MB
    print(f"✅ Video indirildi: {output_path}")
    print(f"📊 Dosya boyutu: {file_size:.1f} MB")
else:
    print(f"❌ Video indirilemedi!")

In [ ]:
# Video özelliklerini kontrol et
import cv2

video_path = "videos/short_video.mp4"
cap = cv2.VideoCapture(video_path)

if cap.isOpened():
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps
    
    print(f"🎬 Video Özellikleri:")
    print(f"   📐 Çözünürlük: {width}x{height}")
    print(f"   🎞️  FPS: {fps}")
    print(f"   📊 Frame sayısı: {frame_count}")
    print(f"   ⏱️  Süre: {duration:.1f} saniye ({duration/60:.1f} dakika)")
    
    cap.release()
else:
    print(f"❌ Video açılamadı: {video_path}")

## 3. YOLO Analizi (Detaylı Bilgi İle)

In [ ]:
# YOLO analizi başlat
print("🎯 YOLO Analizi Başlatılıyor...")
print("=" * 60)

os.chdir('/workspace/line48/src')

yolo_start_time = time.time()

# YOLO komutu
yolo_cmd = [
    "python", "run_analysis.py",
    "--video", "../videos/short_video.mp4",
    "--frame-logic",
    "--detailed-info",
    "--confidence", "0.25",
    "--iou", "0.45", 
    "--imgsz", "1280"
]

print(f"🚀 Komut: {' '.join(yolo_cmd)}")
print("⏳ İşleniyor...")

In [ ]:
# YOLO analizini çalıştır
try:
    yolo_result = subprocess.run(yolo_cmd, capture_output=True, text=True, timeout=1800)
    yolo_time = time.time() - yolo_start_time
    
    print(f"⏱️  YOLO İşleme Süresi: {yolo_time:.1f} saniye")
    
    if yolo_result.returncode == 0:
        print("✅ YOLO analizi başarıyla tamamlandı!")
        print("\n📝 YOLO Çıktısı:")
        print(yolo_result.stdout[-1000:])  # Son 1000 karakter
    else:
        print("❌ YOLO analizi başarısız!")
        print("Hata:", yolo_result.stderr)
        
except subprocess.TimeoutExpired:
    print("⏰ YOLO analizi 30 dakika sonra zaman aşımına uğradı")
except Exception as e:
    print(f"❌ YOLO analizi hatası: {e}")

In [ ]:
# YOLO sonuçlarını kontrol et
import glob
import pandas as pd

print("📊 YOLO Sonuçları:")
print("="*40)

# Output videolarını listele
output_videos = glob.glob('../outputs/*processed*.mp4')
if output_videos:
    latest_output = max(output_videos, key=os.path.getctime)
    print(f"🎬 İşlenmiş video: {latest_output}")
    print(f"📊 Dosya boyutu: {os.path.getsize(latest_output) / (1024*1024):.1f} MB")

# Log dosyalarını kontrol et
result_files = glob.glob('../logs/*results*.csv')
if result_files:
    latest_results = max(result_files, key=os.path.getctime)
    print(f"\n📈 Sonuç dosyası: {latest_results}")
    
    try:
        df_yolo = pd.read_csv(latest_results)
        print("\n🎯 YOLO Tespit Sonuçları:")
        for _, row in df_yolo.iterrows():
            cls = row.get('class', 'unknown')
            safe = row.get('safe_crossings', 0)
            uncertain = row.get('uncertain_crossings', 0)
            total = row.get('total_valid_crossings', 0)
            print(f"  {cls:<10}: Safe={safe}, Uncertain={uncertain}, Total={total}")
    except Exception as e:
        print(f"❌ Sonuç dosyası okunamadı: {e}")
else:
    print("⚠️ Sonuç dosyası bulunamadı")

## 4. SAM Bağımlılıklarını Yükle

In [ ]:
# SAM için gerekli paketleri yükle
print("📦 SAM bağımlılıkları yükleniyor...")
!pip install segment-anything matplotlib Pillow scipy

print("\n✅ SAM bağımlılıkları yüklendi!")

## 5. SAM Analizi (Detaylı Bilgi İle)

In [ ]:
# SAM analizi başlat
print("🔬 SAM Analizi Başlatılıyor...")
print("=" * 60)

sam_start_time = time.time()

# SAM komutu
sam_cmd = [
    "python", "run_sam_analysis.py",
    "--video", "../videos/short_video.mp4",
    "--sam-model", "vit_b",
    "--detailed-info",
    "--download-sam",
    "--confidence", "0.25",
    "--iou", "0.45",
    "--imgsz", "1280"
]

print(f"🚀 Komut: {' '.join(sam_cmd)}")
print("⏳ İşleniyor... (SAM daha yavaş olabilir)")

In [ ]:
# SAM analizini çalıştır
try:
    sam_result = subprocess.run(sam_cmd, capture_output=True, text=True, timeout=3600)  # 60 dakika
    sam_time = time.time() - sam_start_time
    
    print(f"⏱️  SAM İşleme Süresi: {sam_time:.1f} saniye")
    
    if sam_result.returncode == 0:
        print("✅ SAM analizi başarıyla tamamlandı!")
        print("\n📝 SAM Çıktısı:")
        print(sam_result.stdout[-1000:])  # Son 1000 karakter
    else:
        print("❌ SAM analizi başarısız!")
        print("Hata:", sam_result.stderr)
        
except subprocess.TimeoutExpired:
    print("⏰ SAM analizi 60 dakika sonra zaman aşımına uğradı")
except Exception as e:
    print(f"❌ SAM analizi hatası: {e}")

In [ ]:
# SAM sonuçlarını kontrol et
print("🔬 SAM Sonuçları:")
print("="*40)

# SAM output videolarını listele
sam_output_videos = glob.glob('../outputs/*sam_processed*.mp4')
if sam_output_videos:
    latest_sam_output = max(sam_output_videos, key=os.path.getctime)
    print(f"🎬 SAM İşlenmiş video: {latest_sam_output}")
    print(f"📊 Dosya boyutu: {os.path.getsize(latest_sam_output) / (1024*1024):.1f} MB")

# SAM sonuç dosyalarını kontrol et
sam_result_files = glob.glob('../logs/*sam_results*.csv')
if sam_result_files:
    latest_sam_results = max(sam_result_files, key=os.path.getctime)
    print(f"\n📈 SAM Sonuç dosyası: {latest_sam_results}")
    
    try:
        df_sam = pd.read_csv(latest_sam_results)
        print("\n🔬 SAM Tespit Sonuçları:")
        for _, row in df_sam.iterrows():
            cls = row.get('class', 'unknown')
            safe = row.get('safe_crossings', 0)
            uncertain = row.get('uncertain_crossings', 0)
            total = row.get('total_valid_crossings', 0)
            print(f"  {cls:<10}: Safe={safe}, Uncertain={uncertain}, Total={total}")
    except Exception as e:
        print(f"❌ SAM sonuç dosyası okunamadı: {e}")
else:
    print("⚠️ SAM sonuç dosyası bulunamadı")

## 6. Otomatik Karşılaştırma Raporu

In [ ]:
# Otomatik karşılaştırma scripti çalıştır
print("📊 YOLO vs SAM Karşılaştırma Raporu Oluşturuluyor...")
print("=" * 60)

comparison_cmd = [
    "python", "compare_yolo_sam.py",
    "--video", "../videos/short_video.mp4",
    "--detailed-info",
    "--sam-model", "vit_b",
    "--output", "comparison_report_short_video.md"
]

print(f"🚀 Komut: {' '.join(comparison_cmd)}")

try:
    comparison_result = subprocess.run(comparison_cmd, capture_output=True, text=True, timeout=600)
    
    if comparison_result.returncode == 0:
        print("✅ Karşılaştırma raporu oluşturuldu!")
        print("\n📊 Karşılaştırma Çıktısı:")
        print(comparison_result.stdout)
    else:
        print("❌ Karşılaştırma raporu oluşturulamadı!")
        print("Hata:", comparison_result.stderr)
        
except subprocess.TimeoutExpired:
    print("⏰ Karşılaştırma raporu 10 dakika sonra zaman aşımına uğradı")
except Exception as e:
    print(f"❌ Karşılaştırma hatası: {e}")

## 7. Manuel Karşılaştırma ve Analiz

In [ ]:
# Manuel karşılaştırma tablosu oluştur
print("🔍 Manuel Karşılaştırma Analizi")
print("=" * 50)

# YOLO ve SAM sonuçlarını karşılaştır
try:
    # YOLO sonuçları
    yolo_files = glob.glob('../logs/*results*.csv')
    yolo_files = [f for f in yolo_files if 'sam' not in f]  # SAM dosyalarını hariç tut
    
    # SAM sonuçları
    sam_files = glob.glob('../logs/*sam_results*.csv')
    
    if yolo_files and sam_files:
        latest_yolo = max(yolo_files, key=os.path.getctime)
        latest_sam = max(sam_files, key=os.path.getctime)
        
        df_yolo = pd.read_csv(latest_yolo)
        df_sam = pd.read_csv(latest_sam)
        
        print("\n📊 Detay Karşılaştırma Tablosu:")
        print("-" * 70)
        print(f"{'Sınıf':<12} {'YOLO Total':<12} {'SAM Total':<12} {'Fark':<8} {'İyileşme':<10}")
        print("-" * 70)
        
        total_yolo = 0
        total_sam = 0
        
        classes = ['person', 'backpack', 'handbag', 'suitcase']
        
        for cls in classes:
            yolo_row = df_yolo[df_yolo['class'] == cls]
            sam_row = df_sam[df_sam['class'] == cls]
            
            yolo_total = yolo_row['total_valid_crossings'].iloc[0] if len(yolo_row) > 0 else 0
            sam_total = sam_row['total_valid_crossings'].iloc[0] if len(sam_row) > 0 else 0
            
            total_yolo += yolo_total
            total_sam += sam_total
            
            diff = sam_total - yolo_total
            improvement = (diff / yolo_total * 100) if yolo_total > 0 else 0
            
            print(f"{cls:<12} {yolo_total:<12} {sam_total:<12} {diff:+<8} {improvement:+.1f}%")
        
        print("-" * 70)
        overall_diff = total_sam - total_yolo
        overall_improvement = (overall_diff / total_yolo * 100) if total_yolo > 0 else 0
        print(f"{'TOPLAM':<12} {total_yolo:<12} {total_sam:<12} {overall_diff:+<8} {overall_improvement:+.1f}%")
        
        # Performans karşılaştırması
        print("\n⚡ Performans Karşılaştırması:")
        print(f"⏱️  YOLO İşleme Süresi: {yolo_time:.1f} saniye")
        print(f"⏱️  SAM İşleme Süresi: {sam_time:.1f} saniye")
        
        speed_ratio = yolo_time / sam_time if sam_time > 0 else 0
        print(f"🏃 Hız Oranı: SAM, YOLO'dan {1/speed_ratio:.1f}x daha yavaş")
        
        # Sonuç özeti
        print("\n🏆 SONUÇ ÖZETİ:")
        print("=" * 30)
        
        if overall_improvement > 10:
            print("✅ SAM önemli ölçüde daha iyi sonuç verdi!")
        elif overall_improvement > 5:
            print("🟡 SAM hafif iyileşme sağladı")
        elif overall_improvement > 0:
            print("🟢 SAM minimal iyileşme sağladı")
        else:
            print("🔴 YOLO daha iyi performans gösterdi")
            
        if speed_ratio < 0.5:
            print("⚡ YOLO çok daha hızlı - gerçek zamanlı uygulamalar için tercih edilir")
        elif speed_ratio < 0.8:
            print("🏃 YOLO daha hızlı - iyi bir hız/doğruluk dengesi")
        else:
            print("🐌 İşleme hızları karşılaştırılabilir")
            
    else:
        print("❌ Karşılaştırma için yeterli sonuç dosyası bulunamadı")
        
except Exception as e:
    print(f"❌ Manuel karşılaştırma hatası: {e}")

## 8. Dosya Yönetimi ve Temizlik

In [ ]:
# Oluşturulan dosyaları listele
print("📁 Oluşturulan Dosyalar:")
print("=" * 40)

print("\n🎬 İşlenmiş Videolar:")
for video in glob.glob('../outputs/*.mp4'):
    size = os.path.getsize(video) / (1024*1024)
    print(f"  📹 {os.path.basename(video)} ({size:.1f} MB)")

print("\n📊 Log Dosyaları:")
for log in glob.glob('../logs/*.csv'):
    size = os.path.getsize(log) / 1024
    print(f"  📈 {os.path.basename(log)} ({size:.1f} KB)")

print("\n📋 Raporlar:")
for report in glob.glob('*.md'):
    size = os.path.getsize(report) / 1024
    print(f"  📄 {report} ({size:.1f} KB)")

# Toplam disk kullanımı
total_size = 0
for pattern in ['../outputs/*.mp4', '../logs/*.csv', '*.md']:
    for file in glob.glob(pattern):
        total_size += os.path.getsize(file)

print(f"\n💾 Toplam Disk Kullanımı: {total_size / (1024*1024):.1f} MB")

In [ ]:
# Sonuçları zip'le
import shutil
from zipfile import ZipFile

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"yolo_sam_comparison_{timestamp}.zip"

print(f"📦 Sonuçlar zip'leniyor: {zip_filename}")

with ZipFile(zip_filename, 'w') as zipf:
    # İşlenmiş videolar
    for video in glob.glob('../outputs/*.mp4'):
        zipf.write(video, f"videos/{os.path.basename(video)}")
    
    # Log dosyaları
    for log in glob.glob('../logs/*.csv'):
        zipf.write(log, f"logs/{os.path.basename(log)}")
    
    # Raporlar
    for report in glob.glob('*.md'):
        zipf.write(report, f"reports/{report}")

zip_size = os.path.getsize(zip_filename) / (1024*1024)
print(f"✅ Zip oluşturuldu: {zip_filename} ({zip_size:.1f} MB)")
print(f"📥 İndirmek için: scp root@your-vastai-ip:/workspace/line48/src/{zip_filename} ./")

## 9. GPU Performans Özeti

In [ ]:
# Final GPU durumu
print("🎮 Final GPU Durumu:")
print("=" * 30)
!nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total,temperature.gpu,power.draw --format=csv

# PyTorch memory temizle
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    cached = torch.cuda.memory_reserved(0) / 1024**3
    print(f"\n💾 PyTorch GPU Memory:")
    print(f"   Allocated: {allocated:.2f} GB")
    print(f"   Cached: {cached:.2f} GB")
    
    torch.cuda.empty_cache()
    print(f"✅ GPU cache temizlendi")

print(f"\n🎉 Analiz tamamlandı!")
print(f"⏱️  Toplam süre: YOLO={yolo_time:.1f}s + SAM={sam_time:.1f}s = {yolo_time+sam_time:.1f}s")